In [1]:
import pandas as pd
from Bio import SeqIO
from os import path, system
from COG import COGTranslator

In [5]:
df = pd.read_csv("../data/metadata.csv")
df = df[df['is_synecho'] & df['quality_filter']]
cog_out_dir = "../data/ncbi/processing/synecho_cog/"
df['dmnd_out'] = df['proteins_path'].apply(lambda s: path.join(cog_out_dir,
                                                               path.basename(s).split("_genomic")[0] +
                                                               "_dmnd_out.tsv"))
df['cog_fasta'] = df['proteins_path'].apply(lambda s: path.join(cog_out_dir,
                                                               path.basename(s).split("_genomic")[0] +
                                                               "_cog_annotaton.faa"))

In [12]:
cog_path = "/home/vini/data/db/COG/prot2003-2014.fa.dmnd"
cog_ids = pd.read_csv("/home/vini/data/db/COG/cog2003-2014.csv")
cog_ids = cog_ids.iloc[:, :-1]
cog_ids.columns = "domain_id genome_name protein_id protein_length\
                   domain_start domain_end COG_id membership_class".split()
cog_ids = cog_ids.drop_duplicates("domain_id")
cogt = COGTranslator()

def run_diamond(query, db, output, outfmt=6, threads=8, max_target=0):
    command = f"diamond blastp -q {query} --db {db} -o {output} --outfmt {outfmt} -p {threads}"
    if max_target:
        command += f" -k {str(max_target)}"
    
    query_ = path.basename(query)
    print(f"Running Diamond for {query_} with {str(threads)} threads")
    system(command)
    
    if path.isfile(output):
        print(f"Success. Finished diamond for {query}.\nOutput file is {output}")
        return 0
    else:
        return "Failed. Did not create output file."


def parse_seqid(seqid, kind='name'):
    seqid = int(seqid.split("|")[1])
    query = cog_ids.query(f"domain_id == {seqid}")
    if len(query) >= 1:
        cog_code = cog_ids.query(f"domain_id == {seqid}").iloc[0]['COG_id']
        name = cogt.name_from_code(cog_code)
        letter = cogt.letter_from_code(cog_code)
        output = {
            'name': name,
            'code': cog_code,
            'category': letter
        }
        return output[kind]
    else:
        return 0


def description_parser(description, code, category, annotation):
    split = description.split(" # ")
    id_ = split[0]
    gc_cont = split[-1].split(";")[-1]
    category = "".join([i[0] for i in category])
    output = " # ".join([id_, code, category, annotation, gc_cont])
    return output


def process_row(row):
    # Process Diamond output
    dmnd = pd.read_csv(row['dmnd_out'], sep="\t")
    dmnd.columns = 'qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore'.split()
    dmnd.set_index("qseqid", inplace=True)
    dmnd['annotation'] = dmnd['sseqid'].apply(lambda s: parse_seqid(s))
    dmnd['code'] = dmnd['sseqid'].apply(lambda s: parse_seqid(s, kind='code'))
    dmnd['category'] = dmnd['sseqid'].apply(lambda s: parse_seqid(s, kind='category'))
    
    fasta = SeqIO.to_dict(SeqIO.parse(row['proteins_path'], 'fasta'))
    for ix, row_ in dmnd.iterrows():
        description = fasta[ix].description
        code = row_['code']
        category = row_['category']
        annotation = row_['annotation']
        if code != 0:
            fasta[ix].description = description_parser(description, code, category, annotation)
      
    with open(row['cog_fasta'], 'w') as f:
        SeqIO.write([i for i in fasta.values()], f, 'fasta')
        
    print(f"Finished for {row['sample-id']}.")

Loaded 4631 COG names into memory.


In [ ]:
df.apply(lambda row: run_diamond(query=row['proteins_path'], db=cog_path, output=row['dmnd_out'], max_target=1) if not
         path.isfile(row['dmnd_out'] else pass, axis=1)

In [24]:
df.apply(lambda row: process_row(row) if not path.isfile(row['cog_fasta']) else f"Done for {row['cog_fasta']}", axis=1)

KeyError: ('sample-id', 'occurred at index 3')